In [1]:
from reginald.models.setup_llm import setup_llm

from reginald.models.models.llama_index import (
    setup_settings,
    LlamaIndexLlamaCPP,
    set_global_tokenizer,
    compute_default_chunk_size,
)

/Users/rwood/Library/Caches/pypoetry/virtualenvs/reginald-INSspBS7-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/rwood/Library/Caches/pypoetry/virtualenvs/reginald-INSspBS7-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.evaluation import DatasetGenerator
from llama_index.llms.ollama import Ollama

In [3]:
from transformers import AutoTokenizer

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
# response_model = setup_llm(
#     model="llama-index-llama-cpp",
#     model_name="../../../llama-2-7b-chat.Q4_K_M.gguf",  # or hugging face path
#     data_dir="../../data/",  # needs to be path to your data directory
#     which_index="handbook",
# )

In [6]:
ollama_llm = Ollama(model="llama2:7b-chat", request_timeout=60.0)

In [7]:
chunk_size = compute_default_chunk_size(
    max_input_size=4096, k=3
)  # calculate chunk size

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf"
).encode  # load tokenizer
set_global_tokenizer(tokenizer)

In [9]:
settings = setup_settings(
    llm=ollama_llm,
    max_input_size=4096,
    num_output=512,
    chunk_overlap_ratio=0.1,
    chunk_size=chunk_size,
    k=3,
    tokenizer=tokenizer,
)  # these are settings for the storage context

In [10]:
storage_context = StorageContext.from_defaults(
    persist_dir="../../data/llama_index_indices/wikis/"
)

index = load_index_from_storage(
    storage_context=storage_context,
    settings=settings,
)  # load the data index from storage

In [ ]:
index.docstore.docs  # this is the actual documents (I think)

In [12]:
type(
    index.docstore.docs["f1193613-e2ab-4e83-a43e-4749d11e34bb"]
)  # okay so its actually a node vs document?

llama_index.core.schema.TextNode

In [13]:
dataset_generator = DatasetGenerator.from_documents(index.docstore.docs.values())

/Users/rwood/Library/Caches/pypoetry/virtualenvs/reginald-INSspBS7-py3.11/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:213: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [ ]:
dataset_generator.nodes

In [25]:
eval_questions = dataset_generator.generate_questions_from_nodes(
    5
)  # generate questions from the documents

/Users/rwood/Library/Caches/pypoetry/virtualenvs/reginald-INSspBS7-py3.11/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:310: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [26]:
eval_questions

['Great, thank you for providing the context information! Here are 10 questions based on the provided URL that I think would make great questions for a quiz or examination:',
 'What is the name of the research engineering group mentioned in the URL?',
 'According to the document, what is the main focus of the Almanac?',
 'How often is the Almanac published?',
 'Can you identify any key themes or areas of interest mentioned in the document? If so, please list them.']